In [ ]:
import pygame

# --- Config ---
WIDTH, HEIGHT = 600, 600
ROOM_MARGIN = 50          # thickness of the outer wall margin
ROOM_HEIGHT = 150
ROOM_WIDTH = 150
PLAYER_RADIUS = 8
PLAYER_SPEED = 220        # pixels / second
BG_COLOR = (20, 20, 20)
WALL_COLOR = (200, 200, 200)
PLAYER_COLOR = (100, 200, 255)
TEXT_COLOR = (180, 180, 180)
FPS = 60
DOOR_SIZE = 50           # vertical size of the doorway gap in the wall

def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Base game")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 28)

    # Room boundaries (inner playable area)
    left   = ROOM_MARGIN
    right  = WIDTH - ROOM_MARGIN
    top    = ROOM_MARGIN
    bottom = HEIGHT - ROOM_MARGIN

    # Doorway vertical span (centered)
    door_top = HEIGHT // 2 - DOOR_SIZE // 2
    door_bottom = HEIGHT // 2 + DOOR_SIZE // 2

    # Two rooms: 0 has a door on the RIGHT wall, 1 has a door on the LEFT wall
    current_room = 0

    # Start player in the center
    x, y = WIDTH // 2, HEIGHT // 2

    running = True
    while running:
        dt = clock.tick(FPS) / 1000.0  # seconds since last frame

        # --- Events ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # --- Input ---
        keys = pygame.key.get_pressed()
        dx = dy = 0
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            dx -= 1
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            dx += 1
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            dy -= 1
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            dy += 1

        # Normalize diagonal movement
        if dx != 0 and dy != 0:
            inv = 0.70710678
            dx *= inv
            dy *= inv

        # --- Update position ---
        x += dx * PLAYER_SPEED * dt
        y += dy * PLAYER_SPEED * dt

        # --- Room transitions & wall clamping ---

        # Common vertical clamp
        y = max(top + PLAYER_RADIUS, min(y, bottom - PLAYER_RADIUS))

        if current_room == 0:
            # Left wall is solid
            if x < left + PLAYER_RADIUS:
                x = left + PLAYER_RADIUS

            # Right wall has a door; if in door gap and crossing, switch rooms
            if x > right - PLAYER_RADIUS:
                if door_top <= y <= door_bottom:
                    # Go to room 1
                    current_room = 1
                    x = left + PLAYER_RADIUS + 1  # appear just inside room 2
                else:
                    x = right - PLAYER_RADIUS

        elif current_room == 1:
            # Right wall is solid
            if x > right - PLAYER_RADIUS:
                x = right - PLAYER_RADIUS

            # Left wall has a door; if in door gap and crossing, switch rooms back
            if x < left + PLAYER_RADIUS:
                if door_top <= y <= door_bottom:
                    current_room = 0
                    x = right - PLAYER_RADIUS - 1  # appear just inside room 1
                else:
                    x = left + PLAYER_RADIUS

        # --- Draw ---
        screen.fill(BG_COLOR)

        # Draw the room walls, leaving a door gap on the appropriate side
        # Top and bottom walls (full)
        pygame.draw.line(screen, WALL_COLOR, (left, top), (right, top), 3)
        pygame.draw.line(screen, WALL_COLOR, (left, bottom), (right, bottom), 3)

        if current_room == 0:
            # Left wall (solid)
            pygame.draw.line(screen, WALL_COLOR, (left, top), (left, bottom), 3)
            # Right wall (split for door)
            pygame.draw.line(screen, WALL_COLOR, (right, top), (right, door_top), 3)
            pygame.draw.line(screen, WALL_COLOR, (right, door_bottom), (right, bottom), 3)
            room_name = "Room 1"
        else:
            # Right wall (solid)
            pygame.draw.line(screen, WALL_COLOR, (right, top), (right, bottom), 3)
            # Left wall (split for door)
            pygame.draw.line(screen, WALL_COLOR, (left, top), (left, door_top), 3)
            pygame.draw.line(screen, WALL_COLOR, (left, door_bottom), (left, bottom), 3)
            room_name = "Room 2"

        # Draw player
        pygame.draw.circle(screen, PLAYER_COLOR, (int(x), int(y)), PLAYER_RADIUS)

        # Label current room
        label = font.render(room_name, True, TEXT_COLOR)
        screen.blit(label, (10, 10))

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 